In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Install libraries, import them and load data

In [3]:
! pip install pandas plotly dash dash-bootstrap-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 14.8 MB/s eta 0:00:00


In [16]:
# Import libraries
import dash
import dash_bootstrap_components as dbc
from dash import dcc, Input, Output,html      # dcc: dash core component, input and output: could be used for 'callbacks'
import plotly.express as px #  for graph
import pandas as pd

In [18]:
# Loading data
data= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/healthcare.csv')
data.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [19]:
def load_data():
  data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/healthcare.csv')
  data['Billing Amount']=pd.to_numeric(data['Billing Amount'],errors='coerce')
  data['Date of Admission']=pd.to_datetime(data['Date of Admission'])
  data['YearMonth']=data['Date of Admission'].dt.to_period('M')
  return data
data=load_data()


In [20]:
data.shape

(55500, 16)

In [21]:
data.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results,YearMonth
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal,2024-01
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive,2019-08
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal,2022-09
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal,2020-11
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal,2022-09


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55500 entries, 0 to 55499
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Name                55500 non-null  object        
 1   Age                 55500 non-null  int64         
 2   Gender              55500 non-null  object        
 3   Blood Type          55500 non-null  object        
 4   Medical Condition   55500 non-null  object        
 5   Date of Admission   55500 non-null  datetime64[ns]
 6   Doctor              55500 non-null  object        
 7   Hospital            55500 non-null  object        
 8   Insurance Provider  55500 non-null  object        
 9   Billing Amount      55500 non-null  float64       
 10  Room Number         55500 non-null  int64         
 11  Admission Type      55500 non-null  object        
 12  Discharge Date      55500 non-null  object        
 13  Medication          55500 non-null  object    

In [ ]:
num_records=len(data)
avg_billing=data['Billing Amount'].mean()

# Creating a web application

In [23]:
app=dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# App Layout and Design

In [26]:
from logging import debug
app=dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    # Title
    dbc.Row([
        dbc.Col([
            html.H1("Healthcare Dashboard")
        ], width=12, className="text-center my-5")
    ])
    # Hospital Statistics
    dbc.Row([
        dbc.col(html.Div("Total Patient Record:{num_records}",width=15,className="text-center my=3 top-text"))
    ])

])
app.run(debug=True)

#if __name__=="__main__":
 # app.run_server(debug=True)


<IPython.core.display.Javascript object>